# Viewing AIS tracks

This notebook prototypes the use of spatialpandas to select AIS voyage tracks by tapping on the full set of datashaded AIS data.

In [ ]:
import pandas as pd
import numpy as np
import panel as pn
import datetime as dt
import holoviews as hv
from holoviews.operation.datashader import rasterize
from spatialpandas.geometry import PointArray
from spatialpandas import GeoDataFrame
from colorcet import fire

hv.extension('bokeh')

In [ ]:
broadcast = pd.read_csv('../Broadcast.csv', parse_dates=[1])
broadcast.head()

In [ ]:
%%time
broadcast.loc[:, 'x'], broadcast.loc[:, 'y'] = \
    hv.util.transform.lon_lat_to_easting_northing(broadcast.lon,broadcast.lat)

Create a spatially indexed dataframe to allow selection (may take some time for large datasets).

In [ ]:
%%time
sdf = GeoDataFrame({'geometry':PointArray((broadcast.lon, broadcast.lat)),
                    'x':broadcast.x, 'y':broadcast.y,
                    'date_time': broadcast.date_time,
                    'mmsi_id':broadcast.mmsi_id})

In [ ]:
def highlight_tracks(x,y, delta = 0.02):
    path_data = []
    if None not in [x,y]:
        lon, lat = hv.util.transform.easting_northing_to_lon_lat(x,y)
        selection = sdf.cx[lon-delta:lon+delta, lat-delta:lat+delta]
        
        if len(selection) > 0 and len(selection) < 60:
            mmsids = list(selection['mmsi_id'].unique())
            for mmsid in mmsids:
                pathdf = sdf[sdf['mmsi_id']==mmsid]
                pathdf.sort_values(by='date_time')
                coords = list(zip(pathdf['x'], pathdf['y']))
                path_data.append(coords)  
    return hv.Path(path_data).opts(color='red')

Zoom in and tap on a track to view the entire vessel track

In [ ]:
points = rasterize(hv.Points(broadcast, ['x', 'y']))
tiles = hv.element.tiles.ESRI()

overlay = (tiles * points.opts(cmap=fire[180:], width=900, height=500, cnorm='eq_hist', alpha=0.5))
overlay * hv.DynamicMap(highlight_tracks, streams=[hv.streams.Tap()])